In [ ]:
import pydicom as py
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import datetime
import skimage as sk
import sys

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold

#from sklearn.feature_selection import RFECV
from scipy.stats import gmean

#from sklearn import metrics
from sklearn.metrics import make_scorer, confusion_matrix, precision_score, recall_score, accuracy_score, f1_score, roc_auc_score, balanced_accuracy_score, matthews_corrcoef


from sklearn.ensemble import RandomForestClassifier

from scipy.stats import wilcoxon

In [ ]:
root= "the root/path of the folders"

In [ ]:
#Load training, validation and testing data

xtrain_df= pd.read_csv(root+ "classification/"+ "xtrain_df.csv")
ytrain= np.load(root+ "classification/"+ "ytrain.npy",)
print("Done")


xval_df= pd.read_csv(root+ "classification/"+ "xval_df.csv")
yval= np.load(root+ "classification/"+ "yval.npy")
print("Done")


xtest_df= pd.read_csv(root+ "classification/"+ "xtest_df.csv")
ytest= np.load(root+ "classification/"+ "ytest.npy")
print("Done")

print(f"\nTraining size= {xtrain_df.shape}")
print(f"Validation size= {xval_df.shape}")
print(f"Testing size= {xtest_df.shape}")

print(f"\nytrain= {np.unique(ytrain, return_counts= True)}")
print(f"yval= {np.unique(yval, return_counts= True)}")
print(f"ytest= {np.unique(ytest, return_counts= True)}\n")

In [ ]:
# Load the feature-reduced RFECV dataset

xtrain_rfecv= pd.read_csv(root+ "classification/xtrain_rfecv.csv")
xval_rfecv= pd.read_csv(root+ "classification/xval_rfecv.csv")
xtest_rfecv= pd.read_csv(root+ "classification/xtest_rfecv.csv")

In [ ]:
#Load each of the individual columns of each feature groups from the feature-reduced RFECV dataset

pca_fc= (pd.read_csv(root+ "classification/PCA rfecv.csv").Feature).astype(str)
fo_fc= (pd.read_csv(root+ "classification/FO rfecv.csv").Feature).astype(str)
glcm_fc= (pd.read_csv(root+ "classification/GLCM rfecv.csv").Feature).astype(str)
glrlm_fc= (pd.read_csv(root+ "classification/GLRLM rfecv.csv").Feature).astype(str)
glszm_fc= (pd.read_csv(root+ "classification/GLSZM rfecv.csv").Feature).astype(str)
gldm_fc= (pd.read_csv(root+ "classification/GLDM rfecv.csv").Feature).astype(str)
ngtdm_fc= (pd.read_csv(root+ "classification/NGTDM rfecv.csv").Feature).astype(str)

In [ ]:
# Creating the custom scorers

# Custom scorer for True Negative Rate (TNR)
def tnr(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp)



mcc_scorer= make_scorer(matthews_corrcoef)
balanced_accuracy_scorer= make_scorer(balanced_accuracy_score)
precision_scorer = make_scorer(precision_score)
recall_scorer = make_scorer(recall_score)
f1_scorer= make_scorer(f1_score)
roc_auc_scorer= make_scorer(roc_auc_score)
tnr_scorer = make_scorer(tnr)


# CCS function
def compute_ccs(metrics_dict):
    selected_metrics = [
        metrics_dict["Balanced Accuracy"],
        metrics_dict["F1 Score"],
        metrics_dict["ROC_AUC"],
        metrics_dict["MCC"]
    ]
    return gmean([max(0, metric) for metric in selected_metrics])

In [ ]:
def train_and_evaluate(train_x, train_y, val_x, val_y):
    # Store CCS scores across multiple random runs
    val_ccs_scores = []

    #print("\n***********************************************")
    for seed in range(20):  # Loop over 20 different random states
        rfc = RandomForestClassifier(250, random_state=seed, n_jobs=-1, class_weight="balanced")
        rfc.fit(train_x, train_y)

        # Validate
        val_predictions = rfc.predict(val_x)
        val_probabilities = rfc.predict_proba(val_x)[:, 1]  # For ROC AUC

        val_metrics = {
            "Balanced Accuracy": balanced_accuracy_score(val_y, val_predictions),
            "F1 Score": f1_score(val_y, val_predictions),
            "ROC_AUC": roc_auc_score(val_y, val_probabilities),
            "MCC": matthews_corrcoef(val_y, val_predictions)
        }

        val_ccs = (compute_ccs(val_metrics)*100)
        val_ccs_scores.append(val_ccs)


    # Compute final CCS
    #print("***********************************************")
    print(f"Validation CCS = {val_ccs_scores}")
    print(f"Mean validation CCS = {round(np.mean(val_ccs_scores), 3)}")
    return np.array(val_ccs_scores)

In [ ]:
val_rfecv= train_and_evaluate(xtrain_rfecv, ytrain, xval_rfecv, yval)

In [ ]:
                                                        #Ablation study, part-1

In [ ]:
                                #Checking the performance by dropping individual feature sets from the RFECV data
                                            #Do not use PCA as it is not a radiomics feature group.


excluded_ccs= {}

for name, cols in zip(["PCA", "FO", "GLCM", "GLRLM", "GLSZM", "GLDM", "NGTDM"], 
                      [pca_fc, fo_fc, glcm_fc, glrlm_fc, glszm_fc, gldm_fc, ngtdm_fc]):
    
    print("*****************************************")
    print(f"Removed {name}= {len(cols)}")
    
    xtrain_df2= xtrain_rfecv.drop(cols, axis=1)
    xval_df2= xval_rfecv.drop(cols, axis=1)
    xtest_df2= xtest_rfecv.drop(cols, axis=1)
    print("Remaining columns=", np.shape(xtrain_df2)[1])
    
    excluded_ccs[name]= train_and_evaluate(xtrain_df2, ytrain, xval_df2, yval)

In [ ]:
                    #Testing significance of differences of validation CCS between (RFECV) and individual (Leave-one-out) datasets

In [ ]:
for group, excluded_ccs in excluded_ccs.items():
    stat, p_value = wilcoxon(val_rfecv, excluded_ccs)
    print(f"\n\nExcluding {group}:")
    print(f"  Statistic = {stat:.2f}, p-value = {p_value:.4f}")
    if p_value < 0.05:
        print(f" Significant drop in CCS (p < 0.05) - {group} is important")
    else:
        print(f" No significant drop in CCS (p >= 0.05) - {group} may be less critical")

In [ ]:
                                                        #Ablation study, part-2

In [ ]:
                #Checking the performance measures for individual groups addition to the PCA RFECV with statistical tests

for name, cols in zip(["GLSZM features", "GLDM features", "NGTDM features", "GLRLM features", "GLCM features", "FO features"], 
                      [glszm_fc, gldm_fc, ngtdm_fc, glrlm_fc, glcm_fc, fo_fc]):

    features= pca_fc
    l1= len(features)
    
    features= pd.concat([features, cols], axis= 0)
    print("*****************************************")
    print(f"Length of the added {name}= {len(features)-l1}")
    
    val_group= train_and_evaluate(xtrain_rfecv[features], ytrain, xval_rfecv[features], yval)

    
    
    stat, p_value = wilcoxon(val_rfecv, val_group)
    print(f"\nStatistic = {stat:.2f}, p-value = {p_value:.4f}")
    
    print("Mean diff:", np.mean(np.array(val_group) - np.array(val_rfecv)))
    
    if p_value < 0.05:
        print(f"Significant drop in CCS (p < 0.05) - {name} is important.\n")
    else:
        print(f"No significant drop in CCS (p >= 0.05) - {name} may be less critical.\n")

In [ ]:
        #Testing incremental addition of significant feature sets ranked as per their mean validation CCS in (PCA + groups)

features= pca_fc
f= []

for name, cols in zip(["GLRLM", "GLDM", "FO"], [glrlm_fc, gldm_fc, fo_fc]):

    f.append(name)
    
    #l1= len(features)
    features= pd.concat([features, cols], axis= 0)
    print("*****************************************")
    #print(f"Length of the added {name}= {len(features)-l1}")

    print(f"Added to PCA: {f}")
    
    val_group= train_and_evaluate(xtrain_rfecv[features], ytrain, xval_rfecv[features], yval)

    
    stat, p_value = wilcoxon(val_rfecv, val_group)
    print(f"\nStatistic = {stat:.2f}, p-value = {p_value:.4f}")
    print("Mean diff:", np.mean(np.array(val_group) - np.array(val_rfecv)))
    
    if p_value < 0.05:
        print(f"Significant drop in CCS (p < 0.05) - {name} is important.\n")
    else:
        print(f"No significant drop in CCS (p >= 0.05) - {name} may be less critical.\n")

In [ ]:
#Finalize the RFECV dataset with the significant feature groups only: PCA + GLRLM + GLDM + FO

features= pd.concat([pca_fc, glrlm_fc, gldm_fc, fo_fc], axis= 0)
xtrain_reduced_rfecv= xtrain_rfecv[features]
xval_reduced_rfecv= xval_rfecv[features]
xtest_reduced_rfecv= xtest_rfecv[features]

In [ ]:
xtrain_reduced_rfecv.to_csv(root+ "classification/xtrain_reduced_rfecv.csv", index=False)
xval_reduced_rfecv.to_csv(root+ "classification/xval_reduced_rfecv.csv", index=False)
xtest_reduced_rfecv.to_csv(root+ "classification/xtest_reduced_rfecv.csv", index=False)

In [ ]:
                        #SHAP for Feature Importance for statistically significant feature groups: PCA+ GLRLM + GLDM + FO

In [ ]:
# "SHAP Averaging" or "Ensemble SHAP" on training data

# Initialize variables
shap_score = None  # Start with None since we don't know the shape yet
shap_scores = []   # Store SHAP values for all runs


k = 10  # Number of runs

for i in range(k):
    # Train the Random Forest model
    rfc = RandomForestClassifier(250, random_state=i, n_jobs=-1, class_weight="balanced")
    history = rfc.fit(xtrain_reduced_rfecv, ytrain)
    
    # Compute SHAP values
    explainer = shap.TreeExplainer(rfc)
    shap_value = explainer.shap_values(np.array(xtrain_reduced_rfecv), check_additivity=False)  # Returns a list for classification
    

    
    shap_scores.append(shap_value[1])  # Append the current run's SHAP values

    
    # Accumulate SHAP values (initialize shap_score as an array on the first iteration)
    if shap_score is None:
        shap_score = np.array(shap_value[1])  # Initialize as a NumPy array
    else:
        shap_score += np.array(shap_value[1])  # Add to the accumulated SHAP values
    
    print(f"{i+1}) SHAP calculation done.")

In [ ]:
# Absolute average SHAP values across all runs
shap_score= shap_score/k
np.save(root+ "classification/reduced_rfecv_shap_scores.npy", shap_score)

In [ ]:
# Absolute average SHAP values across all runs
sh = np.mean(np.abs(shap_score), axis=0)  # NP.MEAN(axis=0) to average across all rows.

# Display results
print("Averaged SHAP values:", sh)

In [ ]:
#MinMax scaling is used for normalizing the importance scores to bring them to a same range for comparison.

reduced_rfecv_shap= pd.DataFrame({"feature": xtrain_reduced_rfecv.columns, "si": sh})


# Normalize RF importance
reduced_rfecv_shap['si_normalized'] = MinMaxScaler(feature_range=(0, 1)).fit_transform(reduced_rfecv_shap[['si']])

In [ ]:
# Map features to groups

group_mapping = {}


for feat in list(reduced_rfecv_shap['feature']):
    if feat in pca_fc.values:
        group_mapping[feat] = 'PCA'
    elif feat in fo_fc.values:
        group_mapping[feat] = 'FO'
    elif feat in glrlm_fc.values:
        group_mapping[feat] = 'GLRLM'
    elif feat in gldm_fc.values:
        group_mapping[feat] = 'GLDM'
    else:
        group_mapping[feat] = 'Unknown'  # Catch any unmapped features


# Add group column to DataFrame
reduced_rfecv_shap['group'] = reduced_rfecv_shap['feature'].map(group_mapping)


# Aggregate SHAP values by group
group_shap = reduced_rfecv_shap.groupby('group')['si'].sum().sort_values(ascending=False)


# Calculate total SHAP for percentage contribution
total_shap = group_shap.sum()
group_shap_percent = (group_shap / total_shap * 100).round(2)


# Display results
print("\nSHAP Importance by Feature Group:")
print(group_shap)
print("\nPercentage Contribution by Feature Group:")
print(group_shap_percent)


# Optional: Check for unmapped features
unmapped = reduced_rfecv_shap[reduced_rfecv_shap['group'] == 'Unknown']
if not unmapped.empty:
    print(f"\nWarning: {len(unmapped)} features not mapped to any group:", unmapped['feature'].tolist())

In [ ]:
# Save the dataframe to a CSV file
reduced_rfecv_shap.to_csv(root+ "classification/reduced_rfecv_shap.csv", index=False)